In all the directories in this repo I intend to include notebooks where I will (relatively quicly) explain the flow of the code. The idea is to facilitate the navigation and use of the .py scripts. 

The first one would have to do to run the code in the directory `amazon_reviews_classification_without_DL` is to download the amazon reviews. For the example here I have used the 5-core "clothing, shoes and Jewelry" dataset, containing 278,677 reviews. Simply, go [here](http://jmcauley.ucsd.edu/data/amazon/) and download it. 

Once download it place it in your directory of choice and we are ready to go:

In [1]:
import pandas as pd

from pathlib import Path

In [2]:
DATA_PATH = Path('/home/ubuntu/projects/nlp-stuff/datasets/amazon_reviews')
df_org = pd.read_json(DATA_PATH/'reviews_Clothing_Shoes_and_Jewelry_5.json.gz', lines=True)

In [3]:
df_org.overall.value_counts()

5    163240
4     58357
3     30425
2     15463
1     11192
Name: overall, dtype: int64

We can see that there are many 5 stars reviews and not many 1 or 2 starts reviews. Therefore, what I have done before moving any further is to group 1-2 stars reviews into one category and then "rename" all categories so they go from 0 to `num_class`. This is simply because eventually, I am going to use LightGBM, which gets classes in the format [0, num_class) 

In [4]:
# LightGBM expect classes from [0,num_class)
df = df_org.copy()
df['overall'] = (df['overall']-1).astype('int64')

# group reviews with 1 and 2 scores into one class
df.loc[df.overall==0, 'overall'] = 1

# and back again to [0,num_class)
df['overall'] = (df['overall']-1).astype('int64')

In [7]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1KLRMWW2FWPL4,0000031887,"Amazon Customer ""cameramom""","[0, 0]",This is a great tutu and at a really great pri...,3,Great tutu- not cheaply made,1297468800,"02 12, 2011"
1,A2G5TCU2WDFZ65,0000031887,Amazon Customer,"[0, 0]",I bought this for my 4 yr old daughter for dan...,3,Very Cute!!,1358553600,"01 19, 2013"
2,A1RLQXYNCMWRWN,0000031887,Carola,"[0, 0]",What can I say... my daughters have it in oran...,3,I have buy more than one,1357257600,"01 4, 2013"
3,A8U3FAMSJVHS5,0000031887,Caromcg,"[0, 0]","We bought several tutus at once, and they are ...",3,"Adorable, Sturdy",1398556800,"04 27, 2014"
4,A3GEOILWLK86XM,0000031887,CJ,"[0, 0]",Thank you Halo Heaven great product for Little...,3,Grammy's Angels Love it,1394841600,"03 15, 2014"
